# Bibliotecas

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from auxiliar.arrumar_bases import *
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import random

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load(".streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df

def update_sheet(sheet_name,df):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load("/home/ciroshia/Downloads/github/comissao/.streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    worksheet.clear()
    set_with_dataframe(worksheet, df)

    return df

vmb_path = "assets/venda-mensal-bruta_colab-bi_2025-02-11.xlsx"
agendamento_path = "assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

venda_mensal_df = pd.read_excel(vmb_path)
agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")
garantido_df = get_sheetdata("base_garantido")



In [11]:
# Todas as bases

base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
base_lavieen = criar_base_lavieen(base_compilada)
base_comissoes = criar_comissoes(base_compilada)
base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
revenda_df = cria_base_revenda(venda_mensal_df)

/home/ciroshia/Downloads/github/comissao/auxiliar/arrumar_bases.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenda_df["Unidade"] = revenda_df["Unidade"].str.normalize("NFKC").str.strip().str.lower()
/home/ciroshia/Downloads/github/comissao/auxiliar/arrumar_bases.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenda_df["nome_prestadora"] = revenda_df["nome_prestadora"].str.normalize("NFKC").str.strip().str.lower()


# Visualizar

In [ ]:
base_garantido = criar_base_garantido(base_compilada,garantido_df)
colunas = ["nome_prestadora","Unidade","comissao_garantido_total"]
base_garantido[colunas]

,nome_prestadora,Unidade,comissao_garantido_total,tipo_garantido,valor_garantido
0,agda mikie ribeiro mikami,campinas,4845.0,dia,164
1,alessandra cristina silva dos santos,tatuapé,23976.0,dia,999
2,alice saccaro,ipiranga,866.0,mes,293
3,alice saccaro,mooca,591.0,mes,591
4,aline barbosa oliveira saragoça,santo amaro,738.0,dia,738
...,...,...,...,...,...
144,ultraformer iii (santos),santos,511.0,mes,511
145,vanessa teixeira damasceno,mooca,642.0,mes,642
146,veida pereira dias,ipiranga,1594.0,dia,59
147,vivian falheiro de lima,tijuca,9683.0,dia,421


In [8]:
base_garantido = base_compilada.groupby(["nome_prestadora","Unidade","Data"]).agg(comissao_total_dia=('valor_comissao', 'sum')).reset_index()
base_garantido = pd.merge(base_garantido,garantido_df,how="left",on=["nome_prestadora","Unidade"])

base_garantido

,nome_prestadora,Unidade,Data,comissao_total_dia,tipo_garantido,valor_garantido
0,agda mikie ribeiro mikami,campinas,2025-01-06,567.0,dia,R$164.00
1,agda mikie ribeiro mikami,campinas,2025-01-07,59.0,dia,R$164.00
2,agda mikie ribeiro mikami,campinas,2025-01-08,452.0,dia,R$164.00
3,agda mikie ribeiro mikami,campinas,2025-01-09,270.0,dia,R$164.00
4,agda mikie ribeiro mikami,campinas,2025-01-10,163.0,dia,R$164.00
...,...,...,...,...,...,...
1980,vivian falheiro de lima,tijuca,2025-01-29,336.0,dia,R$421.00
1981,vivian falheiro de lima,tijuca,2025-01-30,35.0,dia,R$421.00
1982,vivian falheiro de lima,tijuca,2025-01-31,67.0,dia,R$421.00
1983,yasmin morais rodrigues,santos,2025-01-10,8.0,dia,R$579.00


In [5]:
base_garantido = base_compilada.groupby(["nome_prestadora","Unidade","Data"]).agg(comissao_total_dia=('valor_comissao', 'sum')).reset_index()

base_garantido = pd.merge(base_garantido,garantido_df,how="left",on=["nome_prestadora","Unidade"])

mask_garantido_dia = base_garantido["tipo_garantido"] == "dia"
mask_comissao_dia = base_garantido["comissao_total_dia"] < base_garantido["valor_garantido"]
base_garantido["comissao_garantido_dia"] = base_garantido["comissao_total_dia"]
base_garantido.loc[mask_comissao_dia&mask_garantido_dia,"comissao_garantido_dia"] = base_garantido["valor_garantido"]

groupby_total = base_garantido.groupby(["nome_prestadora","Unidade"]).agg(comissao_garantido_total = ("comissao_garantido_dia","sum")).reset_index()
base_garantido = pd.merge(groupby_total,garantido_df,how="left",on=["nome_prestadora","Unidade"])

mask_garantido_mes = base_garantido["tipo_garantido"] == "mes"
mask_comissao_mes = base_garantido["comissao_garantido_total"] < base_garantido["valor_garantido"]

base_garantido.loc[mask_garantido_mes&mask_comissao_mes,"comissao_garantido_total"] = base_garantido["valor_garantido"]

base_garantido


TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
base_garantido = criar_base_garantido(base_compilada,garantido_df)
prestadoras = base_garantido[["nome_prestadora","Unidade"]].drop_duplicates()

tipo_garantido_options = ["mes", "dia"]
prestadoras["tipo_garantido"] = [random.choice(tipo_garantido_options) for _ in range(len(prestadoras))]
prestadoras["valor_garantido"] = [random.randint(10, 1000) for _ in range(len(prestadoras))]

update_sheet("base_garantido",prestadoras)

In [ ]:
base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
base_lavieen = criar_base_lavieen(base_compilada)
base_comissoes = criar_comissoes(base_compilada)
base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
revenda_df = cria_base_revenda(venda_mensal_df)


base_com_lavieen = juntar_bases(base_comissoes,base_avaliacoes,revenda_df,base_lavieen)

base_com_lavieen.columns

In [ ]:
base_lavieen_filtrada = base_lavieen.loc[base_lavieen["procedimento_padronizado"].str.contains("lavieen",case=False,na=False)]
base_lavieen_groupby = base_lavieen_filtrada.groupby(["nome_prestadora","Unidade"]).agg(quantidade_lavieen=('Data', 'nunique')).reset_index()

comissao_lavieen = 500
base_lavieen_groupby["valor_lavieen"] = comissao_lavieen * base_lavieen_groupby["quantidade_lavieen"]

base_lavieen_groupby